In [ ]:
# Amazon Rekognition Notebook Part 5 - Face Comparisons

In [ ]:
# ------------------------------------------------------------------------------------------------------------------
# Amazon Rekognition is a service that makes it easy to add powerful visual analysis to your applications. 
# Rekognition Image lets you easily build powerful applications to search, verify, and organize millions of images. 
# Rekognition Video lets you extract motion-based context from stored or live stream videos and helps you analyze them.
# Rekognition detects objects, scenes, and faces; extracts text; recognizes celebrities; and identifies inappropriate 
# content in images. It also allows you to search and compare faces. Rekognition Image is based on the same proven, 
# highly scalable, deep learning technology developed by Amazon’s computer vision scientists to analyze billions of 
# images daily for Prime Photos.
# ------------------------------------------------------------------------------------------------------------------

In [1]:
# -------------------------------------------------------------------------
# Import Python Libraries. Press run.
# -------------------------------------------------------------------------

from pyspark.context import SparkContext
import boto3
import requests
import uuid

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7,application_1565109795194_0008,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
# -------------------------------------------------------------------------------------
# Create the Spark Context and a boto3 client for accessing rekognition API's, Press run.
# -------------------------------------------------------------------------------------

sc = SparkContext.getOrCreate()
client = boto3.client('rekognition', region_name='us-east-1')

In [ ]:
# ----------------------------------------------------------------------------------------------------------
# Amazon Rekognition can store information about detected faces in server-side containers known as collections. 
# You can use the facial information that's stored in a collection to search for known faces in images, stored 
# videos, and streaming videos. Let' store the image of Tiger Woods in a new collection.
# Highlight next cell and Press run.
# ----------------------------------------------------------------------------------------------------------

In [3]:
COLLECTION = "rek-collection-1"
client.create_collection(CollectionId=COLLECTION)
client.list_collections()

BUCKET = "ai-operations-111"
KEY = "rekognition/tiger1.jpg"

IMAGE_ID = uuid.uuid4().hex  

def index_face(bucket, key, collection_id, image_id=None, attributes=(), region="us-east-1"):

    response = client.index_faces(
                                 Image={
                                 "S3Object": {
                                 "Bucket": bucket,
                                 "Name": key,
                                 }
                                 },
                                 CollectionId=collection_id,
                                 ExternalImageId=image_id,
                                     DetectionAttributes=attributes,
                                 )
    return response['FaceRecords']


for record in index_face(BUCKET, KEY, COLLECTION, IMAGE_ID):
    face = record['Face']
    print "  FaceId: {}".format(face['FaceId'])
    print "  ImageId: {}".format(face['ImageId'])


  FaceId: 3b27eb96-2eda-4e5f-a803-e675f19d0838
  ImageId: 1bfa2bd0-a37c-328f-b3ef-3d3d035a2ecf

In [ ]:
# ----------------------------------------------------------------------------------------------------------
# Now let's search for Tiger Woods in our collections database using a completely new picture.
# Highlight next cell and Press run.
# ----------------------------------------------------------------------------------------------------------

In [4]:
KEY = "rekognition/search_tiger.jpg"

def search_faces_by_image(bucket, key, collection_id, threshold=80, region="eu-west-1"):
    response = client.search_faces_by_image(
                         Image={
                         "S3Object": {
                         "Bucket": bucket,
                         "Name": key,
                         }
                         },
                         CollectionId=collection_id,
                         FaceMatchThreshold=threshold,
                         )
    return response['FaceMatches']

for record in search_faces_by_image(BUCKET, KEY, COLLECTION):
        face = record['Face']
        print "Matched Face ({}%)".format(record['Similarity'])
        print "  FaceId : {}".format(face['FaceId'])
        print "  ImageId : {}".format(face['ExternalImageId'])

Matched Face (98.2003479004%)
  FaceId : 3b27eb96-2eda-4e5f-a803-e675f19d0838
  ImageId : a8c5f5e30be34d28912f682e0729f1f4

In [ ]:
# ----------------------------------------------------------------------------------------------------------
# Looks like the picture matched. Comparing faces within collections goes a long way in Master Data 
# Management  initiatives.
# Highlight next cell and Press run.
# ----------------------------------------------------------------------------------------------------------

In [ ]:
# ----------------------------------------------------------------------------------------------------------
# Now let's search for Will Smith in our collections database.
# Highlight next cell and Press run.
# ----------------------------------------------------------------------------------------------------------

In [5]:
KEY = "rekognition/will.jpg"

def search_faces_by_image(bucket, key, collection_id, threshold=80, region="eu-west-1"):
    response = client.search_faces_by_image(
                 Image={
                      "S3Object": {
                      "Bucket": bucket,
                      "Name": key,
                 }
                 },
                 CollectionId=collection_id,
                 FaceMatchThreshold=threshold,
                 )
    return response['FaceMatches']

for record in search_faces_by_image(BUCKET, KEY, COLLECTION):
        face = record['Face']
        print "Matched Face ({}%)".format(record['Similarity'])
        print "  FaceId : {}".format(face['FaceId'])
        print "  ImageId : {}".format(face['ExternalImageId'])
else:
    print('This face is not in our database')

This face is not in our database

In [ ]:
# ----------------------------------------------------------------------------------------------------------
# Will Smith does not exist in our collections database.
# Highlight next cell and Press run.
# ----------------------------------------------------------------------------------------------------------

In [6]:
client.delete_collection(CollectionId=COLLECTION)
client.list_collections()

{u'FaceModelVersions': [], 'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': '2a3878d4-312e-48d6-9d65-bf10b4ad3215', 'HTTPHeaders': {'date': 'Tue, 06 Aug 2019 20:12:11 GMT', 'x-amzn-requestid': '2a3878d4-312e-48d6-9d65-bf10b4ad3215', 'content-length': '43', 'content-type': 'application/x-amz-json-1.1', 'connection': 'keep-alive'}}, u'CollectionIds': []}

In [ ]:
# ----------------------------------------------------------------------------------------------------------
# Code for this tutorial can be found on Github. Please follow the link below.\n,
# https://github.com/mkukreja1/datauniversity,
# We hope you found this tutorial from Data University useful.
# Amazon Rekognition is covered in more detail in the Artificial Intelligence module of
# the AWS Big Data Specialty course. Please contact us via email at  datauniversity.ca@gmail.com 
# or call us at 1 647 464 9563   .
# ----------------------------------------------------------------------------------------------------------